# Brett Tully 
## Assignment 3

In [81]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn import svm

from sklearn.svm import SVC
from sklearn.linear_model import SGDRegressor, Lasso, LinearRegression
from sklearn.svm import SVR 


In [82]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Assignment3_BJT/customer_data.csv")

In [83]:
df.isnull().sum()

label       0
id          0
fea_1       0
fea_2     149
fea_3       0
fea_4       0
fea_5       0
fea_6       0
fea_7       0
fea_8       0
fea_9       0
fea_10      0
fea_11      0
dtype: int64

# Filling Null Values

In [84]:
mean_value=df['fea_2'].mean()
  

df['fea_2'].fillna(value=mean_value, inplace=True)

In [85]:
df.isna().sum()

label     0
id        0
fea_1     0
fea_2     0
fea_3     0
fea_4     0
fea_5     0
fea_6     0
fea_7     0
fea_8     0
fea_9     0
fea_10    0
fea_11    0
dtype: int64

In [86]:
df=df.drop(df[['id']],axis=1)

# Assignment Variables

In [87]:
X = df.drop(df[['label']],axis=1)

In [88]:
y = df['label']

# Scaling X values

In [89]:

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

# Train Test Split

In [90]:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.15,random_state=42)

# Strat K-Fold with SVC

In [91]:
SKFold=StratifiedKFold(n_splits=13)
logistic_regressor=LogisticRegression()
svc = svm.SVC()

In [92]:

svc=SVC(kernel='linear')
scores = cross_val_score(svc, X, y, cv=SKFold, scoring='accuracy') 
print(scores)#

[0.79310345 0.79310345 0.79310345 0.79310345 0.8045977  0.8045977
 0.8045977  0.80232558 0.80232558 0.80232558 0.80232558 0.80232558
 0.80232558]


In [93]:
scores_mean = np.mean(scores)
scores_mean

0.8000123373018322

#Repeated Random Sub-Sampling with SVC Classifier

In [94]:
from sklearn.model_selection import ShuffleSplit, cross_val_score

rrandom = ShuffleSplit(n_splits=11,test_size=.20)

result_rrandom=cross_val_score(svc,X,y,cv=rrandom)

result_rrandom

rrandom_score = np.mean(result_rrandom)

print(rrandom_score)


0.7995959595959596


# Random Search with Logistic Regressor

In [95]:
params = {'solver' :['lbfgs', 'liblinear'],
          'multi_class' :['auto', 'ovr', 'multinomial'],
          'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}


clf=RandomizedSearchCV(logistic_regressor,
                       params,
                       cv=8, 
                       return_train_score=True, 
                       n_iter=7,random_state=42) 
clf.fit(X,y)
allscores_random=clf.cv_results_['mean_test_score']
allscores_random = np.mean(allscores_random)

# Grid Search Parameters

In [96]:
from sklearn.linear_model import LinearRegression


param= { 'C':[0.1,1,10,100],
        'kernel':['rbf','linear','poly'], 
      }


 # Grid Search CV with SVC Classifier 

In [97]:


search = GridSearchCV(estimator=svc,
                      param_grid=param,verbose=3,cv=5)

search.fit(X, y)

search.cv_results_
allscores=search.cv_results_['mean_test_score']
print(allscores)
allscores = np.mean(allscores)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5] END .................C=0.1, kernel=rbf;, score=0.800 total time=   0.1s
[CV 2/5] END .................C=0.1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 3/5] END .................C=0.1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 4/5] END .................C=0.1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 5/5] END .................C=0.1, kernel=rbf;, score=0.800 total time=   0.0s
[CV 1/5] END ..............C=0.1, kernel=linear;, score=0.800 total time=   0.0s
[CV 2/5] END ..............C=0.1, kernel=linear;, score=0.800 total time=   0.0s
[CV 3/5] END ..............C=0.1, kernel=linear;, score=0.800 total time=   0.0s
[CV 4/5] END ..............C=0.1, kernel=linear;, score=0.800 total time=   0.0s
[CV 5/5] END ..............C=0.1, kernel=linear;, score=0.800 total time=   0.0s
[CV 1/5] END ................C=0.1, kernel=poly;, score=0.800 total time=   0.0s
[CV 2/5] END ................C=0.1, kernel=poly;

# Pretty Table with scores 

In [98]:
from prettytable import PrettyTable

In [99]:
myTable = PrettyTable(['Cross Validation Techniques','Classifier','Score'])



myTable.add_row(['Stratified K Fold','SVC', scores_mean])
myTable.add_row(['Repeated Random Subsampling','SVC', rrandom_score])
myTable.add_row(['Randomized search', 'Logistic Regressor',allscores_random])
myTable.add_row(['Grid Search','SVC',allscores])
print(myTable)

+-----------------------------+--------------------+--------------------+
| Cross Validation Techniques |     Classifier     |       Score        |
+-----------------------------+--------------------+--------------------+
|      Stratified K Fold      |        SVC         | 0.8000123373018322 |
| Repeated Random Subsampling |        SVC         | 0.7995959595959596 |
|      Randomized search      | Logistic Regressor | 0.7999999999999999 |
|         Grid Search         |        SVC         | 0.7790370370370371 |
+-----------------------------+--------------------+--------------------+
